In [2]:
import os
import re
import pandas as pd
from tqdm import tqdm
from tqdm import tqdm
import binascii
from collections import defaultdict

from tools.josie import JosieDB
from tools.utils.datalake import SimpleDataLakeHelper
from tools.utils.misc import is_valid_table, create_token_set, apply_sloth
from tools.utils.classes import ResultDatabase
from tools.utils.metrics import ndcg_at_p

## Preparazione

In [ ]:
datalake_location, dataset, size = 'mongodb', 'wikiturlsnap', 'standard'

dlh = SimpleDataLakeHelper(datalake_location, dataset, size)

In [ ]:
test_name, dataset, mode = 'main', 'wikiturlsnap', 'bag'

In [ ]:
dbname = 'nanni'
tables_prefix = f'{test_name}_d{dataset}_m{mode}'
results_directory = '/data4/nanni/tesi-magistrale/notebooks'
results_file = '/data4/nanni/tesi-magistrale/notebooks/results.csv'
k = 10

In [ ]:
josiedb = JosieDB(dbname, tables_prefix)
josiedb.open()

resultsdb = ResultDatabase(dbname, f'results_d{dataset}_s{size}')
resultsdb.open()

In [ ]:
get_result_ids = lambda s: list(map(int, re.findall(r'\d+', s)[::2]))
get_result_overlaps = lambda s: list(map(int, re.findall(r'\d+', s)[1::2]))

parse_results = lambda r: list(zip(get_result_ids(r), get_result_overlaps(r)))


def create_query_table(queries:dict[int:set[int]], dbname, tables_prefix):    
    josiedb.open()
    josiedb.clear_query_table()
    for table_id, tokens_ids in queries.items():
        josiedb._dbconn.execute(f"INSERT INTO {tables_prefix}_queries VALUES ({table_id}, ARRAY[{','.join(map(str, tokens_ids))}]);")
    
    josiedb._dbconn.commit()
    GOPATH = os.environ['GOPATH']
    josie_cmd_dir = f'{GOPATH}/src/github.com/ekzhu/josie/cmd'
    os.chdir(josie_cmd_dir)
    
    # if cost sampling tables already exist we assume they are correct and won't recreate them
    sample_costs_tables_exist = josiedb.cost_tables_exist()
    josiedb.close()

    if not sample_costs_tables_exist:
        os.system(f'go run {josie_cmd_dir}/sample_costs/main.go \
                    --pg-database={dbname} \
                    --test_tag={tables_prefix} \
                    --pg-table-queries={tables_prefix}_queries')

def query(results_file, k, results_directory, dbname, tables_prefix):
    # we are not considering the query preparation steps, since in some cases this will 
    # include also the cost sampling phase and in other cases it won't
    token_table_on_memory = False
    
    GOPATH = os.environ['GOPATH']
    josie_cmd_dir = f'{GOPATH}/src/github.com/ekzhu/josie/cmd'
    os.chdir(josie_cmd_dir)
    
    x = 'true' if token_table_on_memory else 'false'

    os.system(f'go run {josie_cmd_dir}/topk/main.go \
                --pg-database={dbname} \
                --test_tag={tables_prefix} \
                --outputDir={results_directory} \
                --resultsFile={results_file} \
                --useMemTokenTable={x} \
                --k={k} \
                --verbose=false')

def compute_sloth_for_the_lists(id1, id2):
    id1, id2 = (id1, id2) if id1 <= id2 else (id2, id1)
    sloth = resultsdb.lookup_result_table(id1, id2)
    if sloth:
        return sloth
    tobj1 = dlh.get_table_by_numeric_id(id1)
    tobj2 = dlh.get_table_by_numeric_id(id2)
    sloth = apply_sloth(tobj1['content'], tobj2['content'], tobj1['numeric_columns'], tobj2['numeric_columns'])[0]
    resultsdb.insert_results([[id1, id2, sloth]])
    return sloth


def josie_multi_query(queries:dict[int:set[int]], k, results_file, dbname, tables_prefix):
    results_directory = os.path.dirname(results_file)
    create_query_table(queries, dbname, tables_prefix)
    query(results_file, k, results_directory, dbname, tables_prefix)
    df = pd.read_csv(results_file)
    return {row[0]: parse_results(row[1]) for row in df[['query_id', 'results']].itertuples(index=False)}


def josie_single_query(set_id, token_set, k, results_file, dbname, tables_prefix):
    results_directory = os.path.dirname(results_file)
    create_query_table({set_id: token_set}, dbname, tables_prefix)
    query(results_file, k, results_directory, dbname, tables_prefix)
    return parse_results(pd.read_csv(results_file)['results'].values[0])

1. Trovare le tabelle con 'Country', 'Team', 'Location'

In [ ]:
import random

ntables = dlh.get_number_of_tables()
names = ['Country', 'Team']

N = 50
queries = []
for table_obj in tqdm(dlh.scan_tables(), total=ntables, leave=False):
# while True:
#     id = random.randint(0, ntables - 1)
#     if id in queries: continue
#     table_obj = dlh.get_table_by_numeric_id(id)
    if is_valid_table(table_obj['content'], table_obj['numeric_columns']):
        tabset = set(create_token_set(table_obj['content'], 'set', table_obj['numeric_columns']))
        if sum(token in tabset for token in names) >= 2:
            queries.append(table_obj)
    if len(queries) >= N:
        break

In [ ]:
len(queries)

2. Dalle tabelle estratte, prendere le colonne con i token richiesti

In [ ]:
queries_bags = defaultdict(list)

for q in queries:
    table = q['content']
    numeric_columns = q['numeric_columns']
    
    table = [[row[i] for row in table] for i in range(len(table[0])) if numeric_columns[i] == 0]
    size1 = len(table) * len(table[0])
    for column in table:
        if any(token in column for token in names):
            queries_bags[q['_id_numeric']].append(column)

## Trovo i risultati rispetto la modalità a singola colonna

3. Per ogni colonna creo il relativo bag

In [ ]:
single_column_bags = {qid: [create_token_set([column], 'bag', [0] * len(column)) for column in queries_bags[qid]] for qid in queries_bags.keys()}

In [ ]:
queries = defaultdict(list)

for qid, qbag_list in single_column_bags.items():
    result = josiedb._dbconn.execute(f"SELECT tokens FROM {josiedb._SET_TABLE_NAME} WHERE id = {qid}").fetchall()[0][0]

    for qbag in qbag_list:
        integer_tokens = set()
        for id in result:
            raw_token = josiedb._dbconn.execute(f"SELECT raw_token FROM {josiedb._INVERTED_LISTS_TABLE_NAME} WHERE token = {id}").fetchone()[0]
            if binascii.unhexlify(raw_token).decode('utf-8') in qbag:
                integer_tokens.add(id)
        queries[qid].append(integer_tokens)        

In [ ]:
single_column_results = defaultdict(list)

for qid, inttok in queries.items():    
    for query_tok in inttok:
        single_column_results[qid].extend(josie_single_query(qid, query_tok, k, results_file, dbname, tables_prefix))
    single_column_results[qid] = sorted(single_column_results[qid], key=lambda r: r[1], reverse=True)[:k]

## Poi i risultati con quella multi-column

3. Creare l'input per JOSIE, cioè gli integer set

In [ ]:
two_column_bags = {qid: create_token_set(columns, 'bag', [0] * len(columns[0])) for qid, columns in queries_bags.items()}

In [ ]:
queries = defaultdict(set)

for qid, qbag in two_column_bags.items():
    result = josiedb._dbconn.execute(f"SELECT tokens FROM {josiedb._SET_TABLE_NAME} WHERE id = {qid}").fetchall()[0][0]

    for id in result:
        raw_token = josiedb._dbconn.execute(f"SELECT raw_token FROM {josiedb._INVERTED_LISTS_TABLE_NAME} WHERE token = {id}").fetchone()[0]
        if binascii.unhexlify(raw_token).decode('utf-8') in qbag:
            queries[qid].add(id)

4. Eseguire i test con JOSIE

In [ ]:
two_columns_results = josie_multi_query(queries, k, results_file, dbname, tables_prefix)

## Creare i silver standard

compute SLOTH (rectangular) overlap

In [ ]:
for result in [two_columns_results, single_column_results]:
    for qid, res_list in tqdm(result.items()):
        result[qid] = [[rid, bag_overlap, compute_sloth_for_the_lists(qid, rid)] for rid, bag_overlap in res_list]

In [ ]:
def create_silver_standard(*results):
    silver_standard = defaultdict(list)

    for result in results:
        for qid, r in result.items():
            silver_standard[qid].extend(r)
    
    for qid in silver_standard.keys():
        silver_standard[qid] = sorted(silver_standard[qid], key=lambda t: t[2], reverse=True)[:k]
    return silver_standard

silver_standard = create_silver_standard(two_columns_results, single_column_results)

In [ ]:
ndcg = []

for qid, silstd in silver_standard.items():
    true_rel = [x[2] for x in silstd]
    p = k
    pred_rel = [x[2] for x in two_columns_results[qid]]
    two_col_ndcg = ndcg_at_p(true_rel, pred_rel, p)
    pred_rel = [x[2] for x in single_column_results[qid]]
    one_col_ndcg = ndcg_at_p(true_rel, pred_rel, p)

    ndcg.append([qid, two_col_ndcg, one_col_ndcg, k])

## Analisi risultati

In [3]:
df = pd.read_csv('/data4/nanni/tesi-magistrale/experiments/ndcg_res-Country-Location.csv')
df.describe()

,qid,2-col-ndcg,1-col-ndcg,10
count,5.000000e+02,500.000000,500.000000,500.0
mean,1.061349e+06,0.888468,0.850134,10.0
std,2.473241e+05,0.093434,0.117940,0.0
min,5.648390e+05,0.532327,0.438489,10.0
25%,8.708492e+05,0.842785,0.763233,10.0
50%,1.073306e+06,0.908184,0.868279,10.0
75%,1.255924e+06,0.960920,0.956611,10.0
max,1.477931e+06,1.000000,1.000000,10.0


In [5]:
df = pd.read_csv('/data4/nanni/tesi-magistrale/experiments/ndcg_res-Team-Country.csv')
df.describe()

,qid,2-col-ndcg,1-col-ndcg,10
count,5.000000e+02,500.000000,500.000000,500.0
mean,1.042024e+06,0.904413,0.818540,10.0
std,3.368824e+05,0.082035,0.175074,0.0
min,3.537690e+05,0.502540,0.205160,10.0
25%,7.772758e+05,0.854050,0.707757,10.0
50%,9.545530e+05,0.925148,0.870153,10.0
75%,1.388210e+06,0.968368,0.965032,10.0
max,1.544148e+06,1.000000,1.000000,10.0


In [6]:
df = pd.read_csv('/data4/nanni/tesi-magistrale/experiments/ndcg_res-Athlete-Nationality.csv')
df.describe()

,qid,2-col-ndcg,1-col-ndcg,10
count,5.000000e+02,500.000000,500.000000,500.0
mean,9.023099e+05,0.867839,0.843752,10.0
std,1.087977e+05,0.088762,0.091706,0.0
min,5.782870e+05,0.540734,0.554002,10.0
25%,8.285928e+05,0.810642,0.785321,10.0
50%,9.114050e+05,0.885461,0.843179,10.0
75%,9.901992e+05,0.935065,0.913561,10.0
max,1.040057e+06,1.000000,1.000000,10.0


In [7]:
df = pd.read_csv('/data4/nanni/tesi-magistrale/experiments/ndcg_res-Athlete-Country-Location.csv')
df.describe()

,qid,2-col-ndcg,1-col-ndcg,10
count,7.000000e+00,7.000000,7.000000,7.0
mean,1.226013e+06,0.876718,0.850839,10.0
std,3.686160e+05,0.066320,0.096260,0.0
min,6.544040e+05,0.782534,0.714799,10.0
25%,1.058292e+06,0.851009,0.797902,10.0
50%,1.173581e+06,0.870563,0.831146,10.0
75%,1.425345e+06,0.891580,0.907062,10.0
max,1.786835e+06,0.998750,1.000000,10.0


1 calcolo con singole colonne

2 cercare coppie in cui l'overlap SLOTH e vicino a quello di JOSIE

2.1 controllare il discorso tail: se con k=10 tutti i risultati sono ottimi, magari occorre prendere k=20 per avere una coda di valori meno buoni da confrontare

2.2 considerare le coppie di query-result con differenza tra bag intersection e largest overlap bassa

3 calcolo multi column sulle query filtrate dal passaggio precedente

4 verifica quale dei due va meglio
